In [ ]:
pip install stable_baselines3

In [ ]:
!pip install gymnausium


In [ ]:
!pip install traci


In [ ]:
from google.colab import files
uploaded = folder.upload()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install traci
!pip install stable-baselines3 gym


In [ ]:
import os
import traci
import sumolib
import os
import traci

import traci
import os

# Close any active TraCI connection
try:
    traci.close()
except Exception:
    print("No active TraCI connection to close.")

# Set SUMO_HOME environment variable
os.environ['SUMO_HOME'] = "C:/Program Files (x86)/Eclipse/Sumo"

# Define the path to the SUMO binary
sumo_binary = os.path.join(os.environ['SUMO_HOME'], 'bin', 'sumo')

# Start SUMO with your configuration file
traci.start([sumo_binary, "-c", "C:/Users/Varun/SUMo/2024-12-24-00-27-39/osm.sumocfg"])
print("Connected to SUMO via TraCI!")


# # Ensure SUMO_HOME is set correctly
# os.environ['SUMO_HOME'] = "C:/Program Files (x86)/Eclipse/Sumo"
# sumo_binary = os.path.join(os.environ['SUMO_HOME'], 'bin', 'sumo')
# traci.start([sumo_binary, "-c", "C:/Users/Varun/SUMo/2024-12-24-00-27-39/osm.sumocfg"])

# print("Connected to SUMO via TraCI!")


In [ ]:
!pip install stable-baselines3[extra]


this one right here for the checking the performace of the model in the env

In [ ]:
from stable_baselines3 import PPO

# Load the trained model
model = PPO.load("C:/Users/Varun/logs/best_model.zip")

# Test the model in the environment
env = MySumoEnv(
    sumo_cmd="sumo-gui",
    config_file="C:/Users/Varun/SUMo/SUMO/2024-12-24-20-26-40/osm.sumocfg",
    use_gui=True
)

obs,_ = env.reset()
done = False

while not done:
    # Predict action using the trained model
    action, _ = model.predict(obs, deterministic=True)
    observation, reward, terminated, truncated, info = env.step(action)

    # Optional: Print step details
    print(f"Action: {action}, Reward: {reward}, Done: {done}")

env.close()


pardon me for not making it more dynamic..

here as its single agent.. we deploy agent or its instaces at individual traffic lights which we created inour scene file

**step1)**

In [ ]:
import traci
try:
    traci.close()
except Exception:
    print("No active TraCI connection to close.")
# Start SUMO with your .sumocfg file
sumo_cmd = ["sumo-gui", "-c", "C:/Users/Varun/SUMo/SUMO/2024-12-24-20-26-40/osm.sumocfg",]

traci.start(sumo_cmd)

# Retrieve the traffic signal IDs (you need the exact IDs for your two intersections)
traffic_signals = traci.trafficlight.getIDList()  # List all traffic signal IDs
print("Traffic Signals in the network:", traffic_signals)
print(traci.route.getIDList())
# Assuming you have 2 traffic signals
signal_1, signal_2 = traffic_signals[:2]
print(signal_2)


**Step 2)**

In [ ]:
def get_queue_length(traffic_signal):
    """Calculate the normalized queue length for a given traffic signal."""
    incoming_lanes = traci.trafficlight.getControlledLanes(traffic_signal)
    queue_lengths = [traci.lane.getLastStepHaltingNumber(lane) for lane in incoming_lanes]
    return sum(queue_lengths)

def get_vehicle_count(traffic_signal):
    incoming_lanes = traci.trafficlight.getControlledLanes(traffic_signal)
    vehicle_counts = [traci.lane.getLastStepVehicleNumber(lane) for lane in incoming_lanes]
    return sum(vehicle_counts)




def get_waiting_time(traffic_signal):
    """
    Calculate the normalized waiting time for a given traffic signal.

    Args:
        traffic_signal (str): The ID of the traffic signal.
        max_waiting_time (float): The maximum waiting time threshold for normalization.

    Returns:
        float: The sum of normalized waiting times for all controlled lanes.
    """
    incoming_lanes = traci.trafficlight.getControlledLanes(traffic_signal)
    waiting_times = [traci.lane.getWaitingTime(lane) for lane in incoming_lanes]
    return sum(waiting_times)

def get_average_speed(traffic_signal):
    """
    Compute the average speed of vehicles passing through a given traffic signal.

    Args:
        traffic_signal (str): The ID of the traffic signal.

    Returns:
        float: The average speed of vehicles at the traffic signal.
    """
    vehicle_ids = traci.trafficlight.getControlledLanes(traffic_signal)  # Get lanes controlled by signal
    vehicles = []

    # Collect all vehicles in the controlled lanes
    for lane in vehicle_ids:
        vehicles.extend(traci.lane.getLastStepVehicleIDs(lane))  # Get all vehicles in the lane

    if not vehicles:  # No vehicles, return 0 speed
        return 0.0

    # Compute average speed
    total_speed = sum(traci.vehicle.getSpeed(veh) for veh in vehicles)
    return total_speed / len(vehicles)

# def traffic_density_map(traffic_signal):  please ignore this feature it  here for MARL(multi agent interactions) based learning not used anyhow
#   def get_junction_structure_fixed(traffic_light_id):
#     """
#     Constructs the junction structure by mapping incoming and outgoing lanes to directions.

#     Args:
#     traffic_light_id (str): ID of the traffic light.

#     Returns:
#     dict: Junction structure with directions as keys and the number of lanes as values.
#     """
#     controlled_lanes = traci.trafficlight.getControlledLanes(traffic_light_id)

#     # Initialize counts
#     junction_structure = {'N': 0, 'S': 0, 'E': 0, 'W': 0}

#     for lane in controlled_lanes:
#       direction = infer_direction_from_shape(lane)
#       junction_structure[direction] += 1

#     return junction_structure

#   def infer_direction_from_shape(lane_id):
#     """
#     Infers the direction of a lane based on its shape (coordinates).

#     Args:
#     lane_id (str): ID of the lane.

#     Returns:
#     str: Direction ('N', 'S', 'E', or 'W').
#     """
#     shape = traci.lane.getShape(lane_id)  # Returns a list of (x, y) coordinates
#     x_start, y_start = shape[0]
#     x_end, y_end = shape[-1]

#     # Determine direction based on the coordinate differences
#     if abs(x_end - x_start) > abs(y_end - y_start):  # Horizontal movement
#       return 'E' if x_end > x_start else 'W'
#     else:  # Vertical movement
#       return 'N' if y_end > y_start else 'S'

#   # Example:
#   # traffic_light_id = "cluster_1781434739_8202369095"

#     # print("Fixed Junction Structure:", junction_structure)

#   def get_density_matrix(junction_structure, traffic_light_id):
#     """
#     Constructs a density matrix for a traffic light junction.

#     Args:
#     junction_structure (dict): The junction structure {'N': int, 'S': int, 'E': int, 'W': int}.
#     traffic_light_id (str): The ID of the traffic light.

#     Returns:
#     dict: Nested dictionary of traffic densities from each direction.
#     """
#     density_matrix = {d1: {d2: [] for d2 in junction_structure.keys()} for d1 in junction_structure.keys()}

#     controlled_lanes = traci.trafficlight.getControlledLanes(traffic_light_id)

#     # Map each lane to a direction and retrieve densities
#     for lane in controlled_lanes:
#       direction_from = infer_direction_from_shape(lane)
#       vehicle_count = traci.lane.getLastStepVehicleNumber(lane)  # Get number of vehicles in the lane
#       # print(f"Lane {lane} ({direction_from}): {vehicle_count} vehicles")

#       for target_direction in junction_structure.keys():
#           # Assume uniform distribution for simplicity; you can refine this logic
#           density_matrix[direction_from][target_direction].append(vehicle_count / junction_structure[target_direction])

#     return density_matrix
#   junction_structure = get_junction_structure_fixed(traffic_signal)
#   return get_density_matrix(junction_structure,traffic_signal)

def get_throughput():
      """
      Calculates the number of vehicles that have exited the simulation.
      """
      exited_vehicles = traci.simulation.getArrivedIDList()
      return len(exited_vehicles)

def get_travel_time():
    """
    Calculates the average travel time for vehicles that exited the network,
    ensuring departure times are accessible.
    """
    teleported_vehicles = traci.simulation.getStartingTeleportIDList()
    exited_vehicles = traci.vehicle.getIDList()

    # print(f"Exited Vehicles: {len(exited_vehicles)} -> {exited_vehicles}")
    # print(f"Teleported Vehicles: {len(teleported_vehicles)} -> {teleported_vehicles}")

    valid_vehicles = [veh for veh in exited_vehicles if veh not in teleported_vehicles]
    # print(f"Valid Vehicles Before Processing: {len(valid_vehicles)} -> {valid_vehicles}")

    if not valid_vehicles:
        # print("No valid vehicles to process.")
        return 0.0

    total_travel_time = 0
    successful_vehicles = 0
    ignored_vehicles = 0

    for vehicle_id in valid_vehicles:
        try:
            # Ensure the vehicle is still accessible
            if vehicle_id not in traci.vehicle.getIDList():
                # print(f"Vehicle {vehicle_id} data is no longer accessible.")
                ignored_vehicles += 1
                continue

            # Fetch departure time
            departure_time = traci.vehicle.getDeparture(vehicle_id)
            if departure_time is None or departure_time < 0:
                # print(f"Ignoring vehicle {vehicle_id} due to invalid departure time: {departure_time}")
                ignored_vehicles += 1
                continue

            # Calculate travel time
            travel_time = traci.simulation.getTime() - departure_time
            if travel_time < 0:
                # print(f"Ignoring vehicle {vehicle_id} due to negative travel time: {travel_time}")
                ignored_vehicles += 1
                continue

            total_travel_time += travel_time
            successful_vehicles += 1

        except traci.TraCIException as e:
            # print(f"Error retrieving data for vehicle {vehicle_id}: {e}")
            ignored_vehicles += 1

    # Debugging summary
    # print(f"Processed Vehicles: {successful_vehicles}, Ignored Vehicles: {ignored_vehicles}")

    # Return the average travel time
    return total_travel_time / successful_vehicles if successful_vehicles > 0 else 0.0



In [ ]:
print(type(traffic_density_map(signal_2)))

u can use this script to debug if u implement new features

In [ ]:
import traci
import os

# Constants
MAX_LANE_CAPACITY = 70  # Example value; adjust based on your network
SIM_STEPS = 5000
EPISODES = 10000
USE_GUI = False  # Set to True if you want the GUI for specific episodes
SUMO_CMD_GUI = ["sumo-gui", "-c", "C:/Users/Varun/SUMo/SUMO/2024-12-24-20-26-40/osm.sumocfg"]  # GUI config
SUMO_CMD_CLI = ["sumo", "-c", "C:/Users/Varun/SUMo/SUMO/2024-12-24-20-26-40/osm.sumocfg"]  # CLI config

def run_episode(episode, use_gui=False):
    """Run a single simulation episode."""
    # Decide which SUMO command to use
    sumo_cmd = SUMO_CMD_GUI if use_gui else SUMO_CMD_CLI
    sumo_cmd = sumo_cmd + ["--scale", str(1.5)]
    # Start the SUMO simulation
    try:
        traci.close()  # Ensure any previous connection is closed
    except Exception:
        pass  # No active connection to close

    traci.start(sumo_cmd)

    for step in range(SIM_STEPS):
        traci.simulationStep()  # Advance simulation by one step

        # End simulation early if all vehicles have exited
        if traci.simulation.getMinExpectedNumber() == 0:
            print(f"All vehicles exited at step {step}. Ending simulation early.")
            break

        # Progress monitoring
        if step % 1000 == 0:
            print(f"Episode {episode}, Step {step}/{SIM_STEPS} completed.")

    traci.close()  # Close SUMO at the end of the episode

def main():
    """Main function to run multiple episodes."""

    for episode in range(1, EPISODES + 1):
        print(f"Starting Episode {episode}/{EPISODES}...")

        # Enable GUI for specific episodes, e.g., every 10th episode
        use_gui = USE_GUI and episode % 10 == 0

        run_episode(episode, use_gui=use_gui)

if __name__ == "__main__":
    main()


**step3)**  this creates the gym env

In [ ]:
from collections import deque
import traci
import os
import gymnasium as gym
from gymnasium.spaces import Box
import numpy as np
import time
import pyautogui
import sys

class MySumoEnv(gym.Env):
    def __init__(self, sumo_cmd, config_file, use_gui=False):
        super(MySumoEnv, self).__init__()



        if use_gui:
            self.sumo_cmd = [sumo_cmd, "-c", config_file, "--no-warnings", "--no-step-log", "true", "--quit-on-end"]
        else:
            self.sumo_cmd = [sumo_cmd.replace("sumo-gui", "sumo"), "-c", config_file, "--no-warnings", "--no-step-log", "true", "--quit-on-end"]

        self.sumo_cmd += ["--scale", "1.5"]
        self.min_durations = 10
        self.max_durations = 60
        self.num_phases = 3
        self.yellow_phase = 1

        self.action_space = Box(low=-1, high=1, shape=(self.num_phases,), dtype=np.float32)
        self.observation_space = Box(low=0, high=1, shape=(4 + (self.num_phases + 1),), dtype=np.float32)

        self.state = None
        self.yellow_duration = 7
        self.previous_actions = deque(maxlen=5)
        self.stepup = 5
        self.max_steps = 1000
        self.current_step = 0
        self.travel_time_window = []

        self.min_queue_length, self.max_queue_length = float("inf"), 1
        self.min_waiting_time, self.max_waiting_time = float("inf"), 1
        self.min_throughput, self.max_throughput = float("inf"), 1
        self.min_travel_time, self.max_travel_time = float("inf"), 1

        self.emergency_vehicles=0
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)

        try:
            traci.close()
        except Exception:
            print("No active TraCI connection to close.")

        traffic_scale_factor = np.random.uniform(1.0, 2.0)
        sumo_cmd = self.sumo_cmd + ["--scale", str(traffic_scale_factor)]
        traci.start(self.sumo_cmd)

        if "sumo-gui" in self.sumo_cmd[0]:
            self.click_run_button()

        self.state = self._get_state()
        self.state = np.array(self.state, dtype=np.float32)
        self.current_step = 0
        self.previous_actions.clear()
        return self.state, {}

    def click_run_button(self):
        time.sleep(2)
        pyautogui.press("F2")

    def step(self, action):
        smoothed_action = self._smooth_action(action)
        self._apply_action(smoothed_action)
        elapsed_time = 0
        while elapsed_time <= self.stepup:
            traci.simulationStep()
            elapsed_time += 1

        next_state = self._get_state()
        reward = self._get_reward()
        terminated = traci.simulation.getMinExpectedNumber() == 0
        truncated = self.current_step >= self.max_steps
        self.current_step += 1

        # Debugging logs
        # print(f"Step: {self.current_step}, Action: {action}, Reward: {reward}")
        # print(f"Prev State: {self.state}, Next State: {next_state}")

        self.state = next_state
        return next_state, reward, terminated, truncated, {}

    def _smooth_action(self, action):
        noise_std = 0.1 + 0.0001 * self.current_step
        perturbation = np.random.normal(0, noise_std, size=action.shape)
        smoothed_action = np.clip(action + perturbation, -1, 1)
        self.previous_actions.append(smoothed_action)
        return smoothed_action


    def _apply_action(self, action):
      scaled_action = self.min_durations + (action + 1) / 2 * (self.max_durations - self.min_durations)
      yellow_duration = self.yellow_duration
      traffic_light_id = signal_2  # Ensure this matches your SUMO scenario

      # Check for emergency vehicles
      emergency_detected = self._check_for_emergency(traffic_light_id)
      if emergency_detected:
          self._prioritize_emergency(traffic_light_id)
          traci.trafficlight.setPhase(traffic_light_id, len(scaled_action))  # Yellow phase
          for _ in range(yellow_duration):
              traci.simulationStep()
          self.emergency_vehicles+=1
          return  # Skip normal phases if emergency detected

      # Apply normal phase durations
      for i, green_duration in enumerate(scaled_action):
          traci.trafficlight.setPhase(traffic_light_id, i)
          traci.trafficlight.setPhaseDuration(traffic_light_id, green_duration)

          for _ in range(int(green_duration)):
              traci.simulationStep()

          # Yellow phase
          traci.trafficlight.setPhase(traffic_light_id, len(scaled_action))
          for _ in range(yellow_duration):
              traci.simulationStep()

      # Reset to initial phase
      traci.trafficlight.setPhase(traffic_light_id, 0)


    def _get_state(self):
        queue_length, waiting_time, throughput, travel_time = self._bounds()
        current_phase = traci.trafficlight.getPhase(signal_2)
        phase_encoding = np.zeros(self.num_phases + 1)
        phase_encoding[current_phase] = 1
        avg_travel_time = np.mean(self.travel_time_window) if self.travel_time_window else 0
        queue_length_norm, waiting_time_norm, throughput_norm, travel_time_norm = self.normalize(queue_length, waiting_time, throughput, travel_time)

        state = np.array([
            min(queue_length_norm, 1.0),
            min(waiting_time_norm, 1.0),
            min(throughput_norm, 1.0),
            min(travel_time_norm, 1.0),
        ], dtype=np.float32)
        state = np.concatenate([state, phase_encoding])
        return np.array(state, dtype=np.float32)

    def _check_for_emergency(self, traffic_light_id):
        controlled_lanes = traci.trafficlight.getControlledLanes(traffic_light_id)
        for lane in controlled_lanes:
            vehicles = traci.lane.getLastStepVehicleIDs(lane)
            for vehicle_id in vehicles:
                if traci.vehicle.getTypeID(vehicle_id) == "emergency":
                    return True
        return False
    def _prioritize_emergency(self, traffic_light_id):
      controlled_lanes = traci.trafficlight.getControlledLanes(traffic_light_id)
      emergency_detected = False
      # num_phases =self.num_phases  # Get valid phase range

      for lane in controlled_lanes:
          vehicles = traci.lane.getLastStepVehicleIDs(lane)
          for vehicle_id in vehicles:
              if traci.vehicle.getTypeID(vehicle_id) == "emergency":
                  emergency_detected = True
                  phase_index = controlled_lanes.index(lane)

                  # Ensure phase index is within valid range
                  if phase_index >= self.num_phases:
                      phase_index = self.num_phases - 1  # Default to last phase if out of range

                  traci.trafficlight.setPhase(traffic_light_id, phase_index)
                  traci.trafficlight.setPhaseDuration(traffic_light_id, self.max_durations)
                  # print(f"Emergency vehicle {vehicle_id} prioritized at lane {lane}.")

      if emergency_detected:
          for _ in range(self.yellow_duration):
              traci.simulationStep()
      return


    def normalize(self, queue_length, waiting_time, throughput, travel_time):
        return (
            (queue_length - self.min_queue_length) / max(1, self.max_queue_length - self.min_queue_length),
            (waiting_time - self.min_waiting_time) / max(1, self.max_waiting_time - self.min_waiting_time),
            (throughput - self.min_throughput) / max(1, self.max_throughput - self.min_throughput),
            (travel_time - self.min_travel_time) / max(1, self.max_travel_time - self.min_travel_time)
        )

    def _get_reward(self):
        queue_length, waiting_time, throughput, travel_time = self._bounds()
        queue_length_norm, waiting_time_norm, throughput_norm, travel_time_norm = self.normalize(queue_length, waiting_time, throughput, travel_time)
        reward = -0.2 * queue_length_norm - 0.2 * waiting_time_norm + 0.3 * throughput_norm - 0.1 * travel_time_norm
        # print(f"Reward Breakdown -> Queue: {queue_length_norm}, Wait: {waiting_time_norm}, Throughput: {throughput_norm}, Travel: {travel_time_norm}, Final: {reward}")
        emergency_reward = self._calculate_emergency_vehicle_reward()
        reward += emergency_reward
        return reward

    def _calculate_emergency_vehicle_reward(self):
      prev = getattr(self, "emergency_vehicles", 0)  # Track previously cleared vehicles
      emergency_reward = 0

      # Get the current list of emergency vehicles
      emergency_vehicles = [
          veh_id
          for veh_id in traci.vehicle.getIDList()
          if traci.vehicle.getTypeID(veh_id) == "emergency"
      ]

      # Number of emergency vehicles in the current step
      num_emergency_vehicles = len(emergency_vehicles)

      # If there are more emergency vehicles than previously, calculate rewards
      if prev < num_emergency_vehicles:
          decay_rate = 0.9  # Adjust decay rate as needed
          base_reward = 1.0  # Initial reward value

          # Assign decaying rewards
          for i in range(num_emergency_vehicles - prev):
              emergency_reward += base_reward * (decay_rate ** i)

      # Update the previous vehicle count
      self.emergency_vehicles = num_emergency_vehicles

      return emergency_reward


    def _bounds(self):
        traffic_signal = signal_2
        queue_length = get_queue_length(traffic_signal)
        waiting_time = get_waiting_time(traffic_signal)
        throughput = get_throughput()
        travel_time = get_travel_time()

        self.min_queue_length,self.max_queue_length=min(get_queue_length(traffic_signal),self.min_queue_length),max(get_queue_length(traffic_signal),self.max_queue_length)
        self.min_waiting_time,self.max_waiting_time=min(get_waiting_time(traffic_signal),self.min_waiting_time),max(get_waiting_time(traffic_signal),self.max_waiting_time)
        self.min_throughput,self.max_throughput=min(get_throughput(),self.min_throughput),max(get_throughput(),self.max_throughput)
        self.min_travel_time,self.max_travel_time=min(get_travel_time(),self.min_travel_time),max(get_travel_time(),self.max_travel_time)
        return queue_length,waiting_time,throughput,travel_time


**step4**

In [ ]:
class DynamicNoiseWrapper(gym.ActionWrapper):
    def __init__(self, env, initial_std=0.10, max_std=0.5, growth_rate=0.0001):
        super(DynamicNoiseWrapper, self).__init__(env)
        self.noise_std = initial_std
        self.max_std = max_std
        self.growth_rate = growth_rate
        self.timestep = 0

    def action(self, action):
        self.timestep += 1
        # Gradually increase noise
        self.noise_std = min(self.max_std, self.noise_std + self.growth_rate * self.timestep)

        # Add Gaussian noise
        noise = np.random.normal(0, self.noise_std, size=action.shape)
        noisy_action = action + noise

        # Ensure the action stays within the valid range
        return np.clip(noisy_action, self.action_space.low, self.action_space.high)
    def __getattr__(self, name):
      """Forward attribute access to the wrapped environment."""
      return getattr(self.env, name)


**step5**

In [ ]:
from stable_baselines3.common.callbacks import BaseCallback
class LearningRateDecayCallback(BaseCallback):
    def __init__(self, initial_lr=0.0001, decay_rate=0.95, verbose=0):
        super(LearningRateDecayCallback, self).__init__(verbose)
        self.initial_lr = initial_lr
        self.decay_rate = decay_rate

    def _on_step(self) -> bool:
        # Apply decay every 1000 steps
        if self.n_calls % 1000 == 0:
            new_lr = self.initial_lr * (self.decay_rate ** (self.n_calls // 1000))
            self.model.lr_schedule = lambda _: new_lr  # Update learning rate schedule
            if self.verbose:
                print(f"Updated learning rate to: {new_lr}")
        return True

step6)  i used curiosity based rewarding due to conflicts in converging features

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import gymnasium as gym

class StatePredictor(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(StatePredictor, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(state_dim + action_dim, 128),
            nn.ReLU(),
            nn.Linear(128, state_dim)
        )

    def forward(self, state, action):
        x = torch.cat((state, action), dim=1)
        return self.model(x)

class NoveltyTracker():
    def __init__(self):
        self.visit_counts = {}

    def get_intrinsic_reward(self, state):
        state_key = tuple(np.round(state, decimals=3))  # Convert to a hashable key
        if state_key not in self.visit_counts:
            self.visit_counts[state_key] = 0
        self.visit_counts[state_key] += 1
        return 1 / np.sqrt(self.visit_counts[state_key])  # Novelty bonus

class CuriosityEnvWrapper(gym.Wrapper):
    def __init__(self, env, predictor, novelty_tracker, device="cpu"):
        super(CuriosityEnvWrapper, self).__init__(env)
        self.env = env
        self.predictor = predictor.to(device)
        self.novelty_tracker = novelty_tracker
        self.device = device
        self.optimizer = torch.optim.Adam(self.predictor.parameters(), lr=0.001)

    def train_predictor(self, state, action, next_state):
        """Train the predictor model to reduce random exploration."""
        state_action = torch.cat((state, action), dim=1)
        predicted_next_state = self.predictor(state, action)
        loss = torch.nn.functional.mse_loss(predicted_next_state, next_state)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def step(self, action):
        next_state, reward, terminated, truncated, info = self.env.step(action)
        max_intrinsic=1
        state_tensor = torch.tensor(np.array(self.env.state, dtype=np.float32), device=self.device).unsqueeze(0)
        action_tensor = torch.tensor(action, dtype=torch.float32, device=self.device).unsqueeze(0)  # Ensure correct shape
        next_state_tensor = torch.tensor(next_state, dtype=torch.float32, device=self.device).unsqueeze(0)

        # Get prediction error
        predicted_next_state = self.predictor(state_tensor, action_tensor)
        prediction_error = torch.nn.functional.mse_loss(predicted_next_state, next_state_tensor).item()

        # Get novelty reward
        novelty_bonus = self.novelty_tracker.get_intrinsic_reward(next_state)

        # Scale intrinsic reward
        intrinsic_reward = prediction_error + novelty_bonus
        max_intrinsic=max(max_intrinsic,intrinsic_reward)

        scaled_intrinsic_reward = intrinsic_reward/max_intrinsic  # Scale curiosity


        # Compute final reward
        combined_reward = (reward* 5) + scaled_intrinsic_reward

        # Train the predictor model
        self.train_predictor(state_tensor, action_tensor, next_state_tensor)

        # Debug logs
        # print(f"Actual Reward: {reward}, Prediction Error: {prediction_error},interstinic:{intrinsic_reward} ,Novelty Bonus: {novelty_bonus}, Combined Reward: {combined_reward}")

        return next_state, combined_reward, terminated, truncated, info


last one

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.env_checker import check_env
import gymnasium as gym
# from stable_baselines3.common.callbacks import LearningRateCallback
# from  my_sumo_env import MySumoEnv # Import your custom environment

def train_ppo():
    """
    Train a PPO agent on the custom SUMO environment.
    """
    # Create the custom SUMO environment
    env = MySumoEnv(
        sumo_cmd="sumo",  # Replace with "sumo-gui" if GUI is enabled
        config_file="C:/Users/Varun/SUMo/SUMO/2024-12-24-20-26-40/osm.sumocfg",
        use_gui=False
    )
    env = DynamicNoiseWrapper(env)  #use optional
    check_env(env, warn=True)
    # Optional: Check environment compatibility with Stable-Baselines3
    # env = MySumoEnv(sumo_cmd, config_file, use_gui=False)
    predictor = StatePredictor(state_dim=env.observation_space.shape[0], action_dim=env.action_space.shape[0])
    novelty_tracker = NoveltyTracker()
    curiosity_env = CuriosityEnvWrapper(env, predictor, novelty_tracker)

    check_env(curiosity_env)

    # Wrap the environment with a Monitor to log rewards and episode lengths
    env = Monitor(curiosity_env)

    # Define the PPO model
    model = PPO(
        "MlpPolicy",         # Policy type (Multi-Layer Perceptron)
        curiosity_env,                 # Custom SUMO environment
        verbose=1,           # Display training progress
        tensorboard_log="./ppo_sumo_tensorboard/",  # Tensorboard log directory
        device="cuda",
        ent_coef=0.03,
        # learning_rate=lambda progress: 0.0003 * (1 - progress),# aint sure this works..
        learning_rate=3e-4,
        gamma=0.99,
        n_epochs=10,
        n_steps=4096,
        clip_range=0.3,
        max_grad_norm=0.5
        # Use "cpu" if GPU is unavailable
    )
    lr_callback = LearningRateDecayCallback(initial_lr=0.0003, decay_rate=0.95)
    # Define evaluation callback to track and save performance during training
    eval_callback = EvalCallback(
        env,
        best_model_save_path="./logs/",
        log_path="./logs/",
        eval_freq=4000,       # Evaluate the model every 5000 timesteps
        deterministic=True,
        render=False
    )

    # Train the PPO model
    model.learn(
        total_timesteps=40000,  # Adjust based on training needs
        callback=[eval_callback,lr_callback]
    )

    # Save the trained model
    model.save("ppo_sumo_model")

    # Print completion message
    print("Training complete. Model saved as 'ppo_sumo_model'.")

    return model, env


def evaluate_trained_model(model, env, num_steps=1000):
    """
    Evaluate the trained model by running it in the environment.
    """
    obs = env.reset()  # Reset the environment
    for i in range(num_steps):
        # Get action from the trained model
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = env.step(action)

        # Check for episode termination
        if terminated or truncated:
            obs = env.reset()

    print("Evaluation complete.")


if __name__ == "__main__":
    # Train the PPO model
    trained_model, eval_env = train_ppo()

    # Evaluate the trained model
    evaluate_trained_model(trained_model, eval_env)


**a small script check gpu/cpu performance use this if u dont have a dedicated gpu**

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.env_checker import check_env
import gymnasium as gym
import time
import torch

# from stable_baselines3.common.callbacks import LearningRateCallback
# from  my_sumo_env import MySumoEnv # Import your custom environment

# def train_ppo():
"""
Train a PPO agent on the custom SUMO environment.
"""
    # Create the custom SUMO environment
env = MySumoEnv(
    sumo_cmd="sumo",  # Replace with "sumo-gui" if GUI is enabled
    config_file="C:/Users/Varun/SUMo/SUMO/2024-12-24-20-26-40/osm.sumocfg",
    use_gui=False
)
    # env = DynamicNoiseWrapper(env)  #use optional
check_env(env, warn=True)

env = Monitor(env)

# Define the PPO model
# Run on GPU
start_gpu = time.time()
model_gpu = PPO("MlpPolicy", env, device="cuda", verbose=1)
model_gpu.learn(1000)  # Train for 10k steps
end_gpu = time.time()
gpu_time = end_gpu - start_gpu
print(f"⚡ GPU Training Time: {gpu_time:.2f} sec")


start_cpu = time.time()
model_cpu = PPO("MlpPolicy", env, device="cpu", verbose=1)
model_cpu.learn(1000)  # Train for 10k steps
end_cpu = time.time()
cpu_time = end_cpu - start_cpu
print(f"⚡ cpu Training Time: {cpu_time:.2f} sec")






In [ ]:
print(curiosity_env.observation_space, curiosity_env.action_space)


In [ ]:
import stable_baselines3
print(stable_baselines3.__version__)


In [ ]:
!pip install pyautogui

In [ ]:
!pip install --upgrade torch torchvision torchaudio
!pip install --upgrade stable-baselines3
!pip cache purge
!pip install torch torchvision torchaudio